# SVM - Election Result Prediction

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

###  EDA

In [6]:
data = pd.read_csv("../data/Loksabha2019.csv")
data.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
0,Telangana,ADILABAD,SOYAM BAPU RAO,1,BJP,Lotus,MALE,52,52.0,ST,12th Pass,"Rs 30,99,414\n ~ 30 Lacs+","Rs 2,31,450\n ~ 2 Lacs+",376892,482,377374,25.330684,35.468248,1489790
1,Telangana,ADILABAD,Godam Nagesh,0,TRS,Car,MALE,0,54.0,ST,Post Graduate,"Rs 1,84,77,888\n ~ 1 Crore+","Rs 8,47,000\n ~ 8 Lacs+",318665,149,318814,21.399929,29.964370,1489790
2,Telangana,ADILABAD,RATHOD RAMESH,0,INC,Hand,MALE,3,52.0,ST,12th Pass,"Rs 3,64,91,000\n ~ 3 Crore+","Rs 1,53,00,000\n ~ 1 Crore+",314057,181,314238,21.092771,29.534285,1489790
3,Telangana,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13030,6,13036,0.875023,1.225214,1489790
4,Uttar Pradesh,AGRA,Satyapal Singh Baghel,1,BJP,Lotus,MALE,5,58.0,SC,Doctorate,"Rs 7,42,74,036\n ~ 7 Crore+","Rs 86,06,522\n ~ 86 Lacs+",644459,2416,646875,33.383823,56.464615,1937690


In [11]:
data.duplicated().sum()

0

In [9]:
data.isna().sum()

STATE                                          0
CONSTITUENCY                                   0
NAME                                           0
WINNER                                         0
PARTY                                          0
SYMBOL                                       245
GENDER                                       245
CRIMINAL\nCASES                              245
AGE                                          245
CATEGORY                                     245
EDUCATION                                    245
ASSETS                                       245
LIABILITIES                                  245
GENERAL\nVOTES                                 0
POSTAL\nVOTES                                  0
TOTAL\nVOTES                                   0
OVER TOTAL ELECTORS \nIN CONSTITUENCY          0
OVER TOTAL VOTES POLLED \nIN CONSTITUENCY      0
TOTAL ELECTORS                                 0
dtype: int64

In [13]:
data_bkup = data.copy()

In [15]:
data = data.drop(['SYMBOL','GENERAL\nVOTES', 'POSTAL\nVOTES', 'TOTAL\nVOTES',
       'OVER TOTAL ELECTORS \nIN CONSTITUENCY',
       'OVER TOTAL VOTES POLLED \nIN CONSTITUENCY'],axis=1)

In [17]:
data.isna().sum()

STATE                0
CONSTITUENCY         0
NAME                 0
WINNER               0
PARTY                0
GENDER             245
CRIMINAL\nCASES    245
AGE                245
CATEGORY           245
EDUCATION          245
ASSETS             245
LIABILITIES        245
TOTAL ELECTORS       0
dtype: int64

In [19]:
data = data.drop('NAME',axis=1)

In [21]:
#removing rows with NOTA
data=data.drop(data[data['PARTY']=='NOTA'].index)

In [23]:
def converter(x):
    try : 
        x=x.split("\n")[0]
        x=x.split()[1]
        x=x.replace(",","")
        x=float(x)
    except:
        x=0
    return x

In [25]:
data['LIABILITIES'] = data['LIABILITIES'].apply(lambda x : converter(x)) 

In [27]:
data['ASSETS'] = data['ASSETS'].apply(lambda x : converter(x)) 

In [29]:
data.head()

,STATE,CONSTITUENCY,WINNER,PARTY,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,TOTAL ELECTORS
0,Telangana,ADILABAD,1,BJP,MALE,52,52.0,ST,12th Pass,3099414.0,231450.0,1489790
1,Telangana,ADILABAD,0,TRS,MALE,0,54.0,ST,Post Graduate,18477888.0,847000.0,1489790
2,Telangana,ADILABAD,0,INC,MALE,3,52.0,ST,12th Pass,36491000.0,15300000.0,1489790
4,Uttar Pradesh,AGRA,1,BJP,MALE,5,58.0,SC,Doctorate,74274036.0,8606522.0,1937690
5,Uttar Pradesh,AGRA,0,BSP,MALE,0,47.0,SC,Post Graduate,133784385.0,22251891.0,1937690


In [31]:
data.isna().sum()

STATE              0
CONSTITUENCY       0
WINNER             0
PARTY              0
GENDER             0
CRIMINAL\nCASES    0
AGE                0
CATEGORY           0
EDUCATION          0
ASSETS             0
LIABILITIES        0
TOTAL ELECTORS     0
dtype: int64

In [33]:
data.dtypes

STATE               object
CONSTITUENCY        object
WINNER               int64
PARTY               object
GENDER              object
CRIMINAL\nCASES     object
AGE                float64
CATEGORY            object
EDUCATION           object
ASSETS             float64
LIABILITIES        float64
TOTAL ELECTORS       int64
dtype: object

### Splitting x and y

In [36]:
y= data['WINNER']
X=data.drop('WINNER',axis=1)

In [38]:
X=pd.get_dummies(X)

### train test split

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=30, stratify=y)

### SVC

In [44]:
from sklearn.svm import SVC  #SVC - Supoprt Vector Classifier
cls=SVC()

In [46]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':['rbf']}

In [48]:
grid=GridSearchCV(cls, parameters,cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(), param_grid={'kernel': ['rbf']})

In [50]:
#to get the best parameter for the model
grid.best_params_

{'kernel': 'rbf'}

In [52]:
svclassifier=SVC(C=10,gamma=0.1,kernel='rbf',probability=True)
svclassifier.fit(X_train,y_train)
ypred=svclassifier.predict(X_test)

In [53]:
#accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,ypred)

0.7326732673267327

In [54]:
#END